In [28]:
import numpy as np

import gt4py.next as gtx
from gt4py.next.iterator.embedded import MutableLocatedField 
from gt4py.next import float64, neighbor_sum, where
from gt4py.next import Dimension, DimensionKind, FieldOffset

In [29]:
def random_field(
    sizes, *dims, low: float = -1.0, high: float = 1.0
) -> MutableLocatedField:
    return gtx.as_field([*dims],
        np.random.default_rng().uniform(
            low=low, high=high, size=sizes
        )
    )

def zero_field(
    sizes, *dims: Dimension, dtype=float
) -> MutableLocatedField:
    return gtx.as_field([*dims], 
        np.zeros(shape=sizes, dtype=dtype)
    )

In [30]:
CellDim = Dimension("C")

## Conditional: where

The `where` builtin works analogously to the numpy version (https://numpy.org/doc/stable/reference/generated/numpy.where.html)

Both require the same 3 input arguments:
- mask: a field of booleans or an expression evaluating to this type
- true branch: a tuple, a field, or a scalar
- false branch: a tuple, a field, of a scalar

Take a simple numpy example, the `mask` here is a condition:

In [44]:
a_np = np.arange(10.0)
b_np = np.where(a_np < 6.0, a_np, a_np*10.0)
print("a_np array: {}".format(a_np))
print("b_np array: {}".format(b_np))

a_np array: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
b_np array: [ 0.  1.  2.  3.  4.  5. 60. 70. 80. 90.]


### **Task**: replicate this example in gt4py

In [38]:
@gtx.field_operator
def fieldop_where(a: gtx.Field[[CellDim], float64]) -> gtx.Field[[CellDim], float64]:
    return where(a < 6.0, a, a*10.0)

@gtx.program
def program_where(a: gtx.Field[[CellDim], float64],
            b: gtx.Field[[CellDim], float64]):
    fieldop_where(a, out=b) 

In [39]:
def test_where():
    a = gtx.as_field([CellDim], np.arange(10.0))
    b = gtx.as_field([CellDim], np.zeros(shape=10))
    program_where(a, b, offset_provider={})
    
    assert np.allclose(b_np, b)

In [40]:
test_where()
print("Test successful")

AttributeError: module 'gt4py.next' has no attribute 'as_field'

## Domain

The same operation can be performed in gt4py by including the `domain` keyowrd argument on `field_operator` call

### **Task**: implement the same operation as above using `domain` instead of `where`

In [32]:
@gtx.field_operator
def fieldop_domain(a: gtx.Field[[CellDim], float64]) -> gtx.Field[[CellDim], float64]:
    return a*10.0

@gtx.program
def program_domain(a: gtx.Field[[CellDim], float64],
            b: gtx.Field[[CellDim], float64]):
    fieldop_domain(a, out=b, domain={CellDim: (0, 5)}) 

In [33]:
def test_domain():
    a = gtx.as_field([CellDim], np.arange(10.0))
    b = gtx.as_field([CellDim], np.zeros(shape=10))
    program_domain(a, b, offset_provider={})
    
    assert np.allclose(b_np, b)

AttributeError: module 'gt4py.next' has no attribute 'as_field'

In [41]:
test_domain()
print("Test successful")

NameError: name 'test_domain' is not defined

## where and domain

A combination of `where` and `domain` is useful in cases when a certain domain exceeds the field size

e.g. a field `a: gtx.Field[[CellDim], float64]` with shape (10,) is applied `domain={CellDim: (0, 13)}`

### **Task**: combine `domain` and `where` to account for extra indices

Edit the code below such that operations on field `a` are performed only up until the 10th index 

In [ ]:
@gtx.field_operator
def fieldop_domain_where(a: gtx.Field[[CellDim], float64]) -> gtx.Field[[CellDim], float64]:
    return a*10.0

@gtx.program
def program_domain_where(a: gtx.Field[[CellDim], float64],
            b: gtx.Field[[CellDim], float64]):
    fieldop_domain_where(a, out=b, domain={CellDim: (0, 13)}) 

In [ ]:
def test_domain_where():
    a = gtx.as_field([CellDim], np.arange(10.0))
    b = gtx.as_field([CellDim], np.zeros(shape=10))
    program_domain_where(a, b, offset_provider={})
    
    assert np.allclose(a_np*10, b[:10])

In [ ]:
test_domain_where()
print("Test successful")